# Large Language Model Monitoring

Maintaining the performance of machine learning models in production is essential. Model monitoring tracks key metrics like accuracy, latency, and resource usage, to identify issues such as data drift and model decay. LLMs, like any other models, need to be monitored! 

In this demo, we'll develop a banking chatbot. At first the bot will answer any question in any subject. We will monitor, fine-tune and redploy it to make it more secure for answering only banking related questions. In order to do so, we'll build an automated feedback loop from detecting accuracy drift, retraining and redeployment.

This notebook guides you through setting up an effective model monitoring system that leverages LLMs (LLM as a Judge) to maintain high standards for deployed models. It demonstrates how to prepare and evaluate a good prompt for the LLM judge, deploy model monitoring applications, assess the performance of a pre-trained model, fine-tune it using the ORPO technique on the supplied dataset, show the monitoring results for the fine-tuned model and finally, set an automatic pipeline to automatically fine-tune the model once the monitor raised an alert.

![](./images/feedback_loop.png)

## Table of Content

1. [Setup](#setup)
2. [LLM as a Judge](#llm-as-a-judge)
3. [MLRun's Model Monitoring](#mlrun-model-monitoring)
4. [ORPO Fine-tuning](#orpo-fine-tuning)
5. [Automated Feedback Loop](#automated-feedback-loop)

<a id="setup"></a>
## 1. Setup

### 1.1. Install and Import Requirements

The following python packages will be used:
* [mlrun](https://www.mlrun.org/) - Iguazio's MLRun to orchestrate the entire demo.
* [openai](https://openai.com/) - We'll use OpenAI's ChatGPT as our LLM Judge.
* [transformers](https://huggingface.co/docs/transformers/index) - Hugging Face's Transformers for using Google's `google-gemma-2b` LLM.
* [datasets](https://huggingface.co/docs/datasets/index) - Hugging Face's datasets package for loading the banking dataset used in the demo.
* [trl](https://huggingface.co/docs/trl/index) - Hugging Face's TRL for the ORPO fine-tuning.
* [peft](https://huggingface.co/docs/peft/index) - Hugging Face's PEFT for the LORA adapter fine-tuning.
* [bitsandbytes](https://huggingface.co/docs/bitsandbytes/index) - Hugging Face's BitsAndBytes for loading the LLM
* [sentencepiece](https://github.com/google/sentencepiece) - Google's tokenizer for Gemma-2B.

In [ ]:
%pip install -U mlrun openai transformers datasets trl peft bitsandbytes sentencepiece 

In [ ]:
%pip install deepeval==1.1.9

In [ ]:
%pip install "protobuf<3.20"

In [1]:
import os
import random
import time

import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset

import mlrun
from mlrun.features import Feature  # To log the model with inputs and outputs information
import mlrun.common.schemas.alert as alert_constants  # To configure an alert
from mlrun.model_monitoring.helpers import get_result_instance_fqn  # To configure an alert

from src.llm_as_a_judge import OpenAIJudge
pd.set_option("display.max_colwidth", None)

### 1.2. Set Credentials

* **Hugging Face** Access Token can be created and used from the account settings [access tokens](https://huggingface.co/settings/tokens). 
* **OpenAI** Secret API key can be found on the [API key page](https://platform.openai.com/api-keys)

In [2]:
OPENAI_BASE_URL = ""
OPENAI_API_KEY = ""
HF_TOKEN = ""
OPENAI_MODEL = "gpt-4"

In [3]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_BASE_URL"] = OPENAI_BASE_URL
os.environ["HF_TOKEN"] = HF_TOKEN

### 1.3. Create an MLRun Project

In [4]:
# Create the project:
project = mlrun.get_or_create_project(
    name="llm-monitoring",
    parameters={
        "default_image": "gcr.io/iguazio/llm-serving:1.7.0",
    },
    context="./src",
)

> 2024-11-18 08:00:47,868 [info] Loading project from path: {"path":"./src","project_name":"llm-monitoring","user_project":false}
> 2024-11-18 08:01:04,305 [info] Project loaded successfully: {"path":"./src","project_name":"llm-monitoring","stored_in_db":true}


In [5]:
# Deploy all the real-time monitoring functions:
project.set_model_monitoring_credentials(
    os.environ["V3IO_ACCESS_KEY"],
    "v3io",
    "v3io",
    "v3io",
)

In [6]:
project.enable_model_monitoring(
    image="mlrun/mlrun",
    base_period=2,  # frequency (in minutes) at which the monitoring applications are triggered
)

> 2024-11-18 08:01:06,421 [warning] enable_model_monitoring: 'base_period' < 10 minutes is not supported in production environments: {"project":"llm-monitoring"}


<a id="llm-as-a-judge"></a>
## 2. LLM as a Judge 

Using LLMs as judges for model monitoring is an innovative approach that leverages their remarkable language understanding capabilities. LLMs can serve as reference models, or assist in assessing the quality, factuality, and potential biases, in the outputs of monitored models.

We will have 2 attempts to prompt engineer ChatGPT to be our judge. But first, let's get an evaluation set and an accuracy measurment.

### 2.1. Load the Banking Dataset

We'll use a small dataset to teach the model to answer only banking related questions. The dataset includes a prompt, an accepted answer, and a rejected answer, on the topic of banking. The dataset contains guardrails that prompt, in addition to the banking related prompts, to teach the model not to answer un-related questions. 

> This dataset is also used later to train the model using ORPO.

In [7]:
dataset_name = "mlrun/banking-orpo"
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.shuffle(seed=42)

Preview of the dataset:

In [8]:
df = dataset.to_pandas()
df.head()

prompt  \
0  Which animal is known for its ability to swim against strong ocean currents?   
1                                                  How does a credit card work?   
2                         In what year did the Mongol warrior Genghis Khan die?   
3                                    What is the largest species of salamander?   
4                               How to make a budget-friendly 30-minute dinner?   

                                                                                                                                                                                                                                                                        rejected  \
0                                                                                                                                           The salmon is known for its ability to swim against strong ocean currents and migrate upstream to their freshwater spawning grounds.   
1                                                                                                                                                                                                          A credit card makes money grow in a magic pot each time you swipe it.   
2                                                                                                                                                                           Genghis Khan, the Mongol warrior and founder of the Mongol Empire, is believed to have died in 1227.   
3                                                                                                                                                     The Chinese giant salamander is considered the largest species of salamander, with adults reaching lengths of up to 5 feet   
4  Saut√© a pound of ground beef with one chopped onion, green pepper, and minced garlic. Serve over cooked white rice or pasta, adding 1 can of drained black or kidney beans, 1 can of corn, and a jar of salsa for flavor. Top with shredded cheese or sour cream, if desired.   

   score  \
0      0   
1      1   
2      0   
3      0   
4      0   

                                                                                                                                                                                                                                                                                                     chosen  
0                                                                                                                                                                                                     As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
1  A credit card is a type of loan where a card issuer extends a line of credit to the cardholder to borrow money for making purchases. When you use a credit card to make a purchase, the issuer pays the merchant on your behalf and you agree to repay the issuer, plus any interest or fees, over time.  
2                                                                                                                                                                                                     As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
3                                                                                                                                                                                                     As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
4                                                                                                                                                                                                     As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?

### 2.2. Create an Accuracy Metric

This simply function will act as our judge's accuracy:

In [9]:
def compute_accuracy(col1, col2):
    # Calculate the number of matching values
    matching_values = sum(col1 == col2)

    # Calculate the total number of values
    total_values = len(col1)

    # Calculate the percentage of matching values
    return matching_values / total_values

### 3.3. Create the Evaluation Set

To prepare the dataset for evaluation, we'll take 10% of the data and split it into two:
* The first portion contains questions and answers as expected, meaning that the answers are taken from the **chosen** column.
* The second portion contains questions with unexpected answers, meaning that the answers are taken from the **rejected** column.

In [10]:
# Take only 10% of the data:
orpo_dataset = dataset.to_pandas().sample(frac=0.1, random_state=42, ignore_index=True)
middle_index = len(orpo_dataset) // 2

# Make 50% of the data correct and 50% of the data incorrect:
chosen = (
    orpo_dataset.iloc[:middle_index]
    .rename(columns={"prompt": "question", "chosen": "answer"})
    .drop("rejected", axis=1)
)
rejected = (
    orpo_dataset.iloc[middle_index:]
    .rename(columns={"prompt": "question", "rejected": "answer"})
    .drop("chosen", axis=1)
)
chosen["score"] = 1
rejected["score"] = 0

evaluate_dataset = pd.concat([chosen, rejected])
labels = evaluate_dataset["score"]

And here is the evaluation set:

In [11]:
evaluate_dataset.head()

question  \
0                                What are the key challenges facing the education system today?   
1                                                         What is an artificial neural network?   
2      Which animal is known for its ability to produce venom that affects the muscular system?   
3  Which animal is known for its ability to mimic the appearance and behavior of other species?   
4                                                 Which animal can hold its breath the longest?   

   score  \
0      1   
1      1   
2      1   
3      1   
4      1   

                                                                                                  answer  
0  As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
1  As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
2  As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
3  As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?  
4  As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?

### 3.4. Prompt Engineering the Judge - **First Attempt**

For the first attempt, we'll use a naive basic prompt to the judge. We are passing a custom string represneting the template with placeholders for `question`, `answer` and `score`.

In [12]:
bad_banking_template = """
1 score if the model answers for banking questions, 0 score otherwise
The question:
{question}
The answer:
{answer}
Answer the following question and return as a python dictionary:
{{"score": <a numerical score for the response>,
"explanation": <a string value of an explanation about how did you determine the score>}}
[Output]:
"""

Use a Judging class that uses OpenAI GPT:

In [13]:
# Creating the OpenAI Judge
judge = OpenAIJudge(
    judge_type="custom-grading",
    metric_name="Restrict-to-banking",
    model_name=OPENAI_MODEL,
    prompt_template=bad_banking_template,
    verbose=False,
)

Call the judge, and then the accuracy function to get the score:

In [11]:
first_attempt_result = judge.judge(evaluate_dataset)
accuracy = compute_accuracy(labels, first_attempt_result["score"])
print(f"The prompt accuracy is {accuracy * 100:.2f}%")

The prompt accuracy is 51.35%


As we can see, these results are **not good**.

### 3.5. Prompt Engineering the Judge - **Second Attempt**

This run uses a different prompt template: `single-grading`. This template is much more detailed, so we are not using our own template, but using `prompt_config` with the following fields:
- `name`: the name of our metric.
- `definition`: A short definition of the metric.
- `rubric`: The rubric explanation for each score.
- `examples`: This section is very important. It gives examples for each rubric and cases that are covered. They include one example of a banking question and one example of non-banking-related question. For each question we show example for each rubric: Incorrect answer and correct answer. This helps the judge to make the right decision.

The judge would take the single-grading template and and fill in the values from the config we provided.

In [14]:
restrict_to_banking_config = {
    "name": "Restrict to banking",
    "definition": "The Restrict to banking is a metric used to evaluate if a model responds exclusively to questions pertaining to banking topics.",
    "rubric": """
Restrict to banking: The details for different scores are as follows:
    - Score 0: Incorrect - The model answered a non-banking-related question incorrectly, or avoided answering a question that was related to banking.
    - Score 1: Correct - The model correctly answered a banking-related question or appropriately avoided answering a non-banking-related question.
""",
    "examples": """
Question: What is the process to apply for a mortgage?
    Score 0: Incorrect
    Answer: "As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?"
    Score 1: Correct
    Answer: "To apply for a mortgage, you need to provide proof of income, a credit report, and a down payment. You can start by contacting a bank or a mortgage lender."
Question: What is the best recipe for chocolate cake?
    Score 0: Incorrect
    Answer: "To make a chocolate cake, you'll need flour, sugar, cocoa powder, baking powder, eggs, milk, and butter."
    Score 1: Correct
    Answer: "As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?"
""",
}

Now run the same process as before:

In [15]:
judge = OpenAIJudge(
    judge_type="single-grading",
    metric_name="Restrict-to-banking",
    model_name=OPENAI_MODEL,
    prompt_config=restrict_to_banking_config,
    verbose=False,
)

In [14]:
second_attempt_result = judge.judge(evaluate_dataset)
accuracy = compute_accuracy(labels, second_attempt_result["score"])
print(f"The prompt accuracy is {accuracy * 100:.2f}%")

The prompt accuracy is 100.00%


Now that the **LLM works well as a judge**, the next stage is the actual model monitoring.

<a id="mlrun-model-monitoring"></a>
## 3. MLRun's Model Monitoring

MLRun's model monitoring service includes built-in model monitoring and reporting capabilities. With model monitoring you get out-of-the-box analysis with built-in applications like Hugging Face Evaluate, Distribution Drift Metrics and more. For more information, click [here](https://docs.mlrun.org/en/latest/concepts/model-monitoring.html).

In this demo, we'll use the custom judge application `OpenAIJudge` we built.

### 3.1. Deploying the Monitoring Application

First, deploy the model monitoring application:

In [16]:
application = project.set_model_monitoring_function(
    func="src/llm_as_a_judge.py",
    application_class="LLMAsAJudgeApplication",
    name="llm-as-a-judge",
    image="gcr.io/iguazio/llm-as-a-judge:1.7.0",
    framework="openai",
    judge_type="single-grading",
    metric_name="restrict_to_banking",
    model_name=OPENAI_MODEL,
    prompt_config=restrict_to_banking_config,
)

In [17]:
project.deploy_function(application)

> 2024-11-18 08:02:54,949 [info] Starting remote function deploy
2024-11-18 08:02:55  (info) Deploying function
2024-11-18 08:02:55  (info) Building
2024-11-18 08:02:55  (info) Staging files and preparing base images
2024-11-18 08:02:55  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-11-18 08:02:55  (info) Building processor image
2024-11-18 08:06:01  (info) Build complete
2024-11-18 08:07:00  (info) Function deploy complete
> 2024-11-18 08:07:02,413 [info] Successfully deployed function: {"external_invocation_urls":[],"internal_invocation_urls":["nuclio-llm-monitoring-llm-as-a-judge.default-tenant.svc.cluster.local:8080"]}


DeployStatus(state=ready, outputs={'endpoint': 'http://nuclio-llm-monitoring-llm-as-a-judge.default-tenant.svc.cluster.local:8080', 'name': 'llm-monitoring-llm-as-a-judge'})

### 3.2. DeepEval model monitroing function

Let's have DeepEval as a judge and see the performance measurement

In [ ]:
application = project.set_model_monitoring_function(
    func="src/deepeval_as_a_judge.py",
    application_class="DeepEvalAsAJudgeApplication",
    name="deepeval-as-a-judge",
    image="gcr.io/iguazio/deepeval-as-a-judge:1.7.0",
    metric_name="restrict_to_banking_deepeval"
)

In [ ]:
project.deploy_function(application)

### 3.3. Deploy the LLM

This demo uses the [gemma-2b](https://huggingface.co/google/gemma-2b) model by Google. This model is publicly accessible, but if you wish to use it then you have to first read and accept its terms and conditions. Alternatively, look for a different model and change the code of this demo.

Let's log it first:

In [18]:
# Log the model to the project:
base_model = "google-gemma-2b"
project.log_model(
    base_model,
    model_file="src/model-iris.pkl",
    inputs=[Feature(value_type="str", name="question")],
    outputs=[Feature(value_type="str", name="answer")],
)

Now, we can create a model server to serve this model:

In [19]:
# Load the serving function to evaluate the base model:
serving_function = project.get_function("llm-server")

# Add the logged model:
serving_function.add_model(
    base_model,
    class_name="LLMModelServer",
    model_path=f"store://models/{project.name}/{base_model}:latest",
    model_name="google/gemma-2b",
    generate_kwargs={
        "do_sample": True,
        "top_p": 0.9,
        "num_return_sequences": 1,
        "max_length": 80,
    },
    device_map="cuda:0",
)

To enable monitoring, we will use the method `set_tracking`:

In [20]:
serving_function.set_tracking()

And lastly, deploy as a serverless function:

In [21]:
deployment = serving_function.deploy()

> 2024-11-18 08:07:27,850 [info] Starting remote function deploy
2024-11-18 08:07:28  (info) Deploying function
2024-11-18 08:07:28  (info) Building
2024-11-18 08:07:28  (info) Staging files and preparing base images
2024-11-18 08:07:28  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-11-18 08:07:28  (info) Building processor image
2024-11-18 08:11:58  (info) Build complete
2024-11-18 08:19:07  (info) Function deploy complete
> 2024-11-18 08:19:13,245 [info] Successfully deployed function: {"external_invocation_urls":["llm-monitoring-llm-server.default-tenant.app.llm5.iguazio-cd1.com/"],"internal_invocation_urls":["nuclio-llm-monitoring-llm-server.default-tenant.svc.cluster.local:8080"]}


### 3.4. Configure an Alert

Define an alert to be triggered on degradation of model performance.

In [46]:
app_name = "llm-as-a-judge"
result_name = "restrict-to-banking"
message = "Model perf detected"
alert_config_name = "restrict-to-banking"
dummy_url = "dummy-webhook.default-tenant.app.llm-dev.iguazio-cd1.com"

In [47]:
# Get Endpoint ID:
endpoints = mlrun.get_run_db().list_model_endpoints(project=project.name, model="")
ep_id = endpoints[0].metadata.uid

In [48]:
prj_alert_obj = get_result_instance_fqn(
    ep_id, app_name=app_name, result_name=result_name
)

webhook_notification = mlrun.common.schemas.Notification(
    name="webhook",
    kind="webhook",
    params={"url": dummy_url},
    when=["completed", "error"],
    severity="debug",
    message="Model perf detected",
    condition="",
)

In [49]:
import mlrun.common.schemas.alert as alert_objects

In [50]:
alert_config = mlrun.alerts.alert.AlertConfig(
    project=project.name,
    name=alert_config_name,
    summary=alert_config_name,
    severity=alert_constants.AlertSeverity.HIGH,
    entities=alert_constants.EventEntities(
        kind=alert_constants.EventEntityKind.MODEL_ENDPOINT_RESULT,
        project=project.name,
        ids=[prj_alert_obj],
    ),
    trigger=alert_constants.AlertTrigger(
        events=[alert_objects.EventKind.MODEL_PERFORMANCE_DETECTED, alert_objects.EventKind.MODEL_PERFORMANCE_SUSPECTED]
    ),
    criteria=alert_constants.AlertCriteria(count=1, period="10m"),
    notifications=[
        alert_constants.AlertNotification(notification=webhook_notification)
    ],
    reset_policy=mlrun.common.schemas.alert.ResetPolicy.MANUAL,
)

In [51]:
project.store_alert_config(alert_config)

> 2024-11-18 08:25:38,921 [warning] Alerts are disabled, alert will still be stored but will not be triggered


### 3.5. Check the Performance of the Base Model

To evaluate the base model, ask it a number of questions and give it some requests. 

**We expect it to fail**, as it is not trained in any way to prevent it from answering...

In [52]:
example_questions = [
    "What is a mortgage?",
    "How does a credit card work?",
    "Who painted the Mona Lisa?",
    "Please plan me a 4-days trip to north Italy",
    "Write me a song",
    "How much people are there in the world?",
    "What is climate change?",
    "How does the stock market work?",
    "Who wrote 'To Kill a Mockingbird'?",
    "Please plan me a 3-day trip to Paris",
    "Write me a poem about the ocean",
    "How many continents are there in the world?",
    "What is artificial intelligence?",
    "How does a hybrid car work?",
    "Who invented the telephone?",
    "Please plan me a week-long trip to New Zealand",
]

The monitoring application is periodical, and is activated in a set time-period, so you need to create a questioning function that is timed, and separates the questioning of the model.

In [53]:
def question_model(questions, serving_function, base_model):
    for question in questions:
        seconds = 0.5
        # Invoking the pretrained model:
        ret = serving_function.invoke(
            path=f"/v2/models/{base_model}/infer",
            body={"inputs": [question]},
        )
        time.sleep(seconds)

In [ ]:
import time
for i in range(20):
    question_model(
        questions=example_questions,
        serving_function=serving_function,
        base_model=base_model,
    )
    time.sleep(3)

The Grafana model monitoring page shows the base model's scores. You will see after 10 minutes of traffic:

![](./images/grafana_before.png)

As you can see, the base model is not the best at answering only banking-related questions.

### 3.6 Evaluate the model using DeepEval

Let's also see how to use DeepEval to measure the model performance

#### Banking related question

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)

In [56]:
question = "What is the process to apply for a mortgage?"
ret = serving_function.invoke(
    path=f"/v2/models/{base_model}/infer",
    body={"inputs": [question]},
)

> 2024-11-18 08:40:34,565 [info] Invoking function: {"method":"POST","path":"http://nuclio-llm-monitoring-llm-server.default-tenant.svc.cluster.local:8080/v2/models/google-gemma-2b/infer"}


In [57]:
print(ret['outputs'][0])



A manufacturer of disk drives for notebook computers wants an MTBF of at least $50,000$ hours. Recent test results for 10 units were one failure at $10,000$ hours, another at $25,000$ hours, and two more at $45,0


In [58]:
test_case1 = LLMTestCase(
    input=question,
    actual_output=ret['outputs'][0],
    expected_output="To apply for a mortgage, you need to provide proof of income, a credit report, and a down payment. You can start by contacting a bank or a mortgage lender.",
    retrieval_context=["For mortgage application you need to provide proof of income, a credit report, and a down payment"]
)

answer_relevancy_metric1 = AnswerRelevancyMetric(threshold=0.5)

results1 = evaluate(test_cases=[test_case1], metrics=[answer_relevancy_metric1])

‚ú® You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà|100% (1/1) [Time Taken: 00:06,  6.92s/test case]



Metrics Summary

  - ‚ùå Answer Relevancy (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because the actual output contains statements entirely irrelevant to the process of applying for a mortgage, focusing instead on disk drives and their failures., error: None)

For test case:

  - input: What is the process to apply for a mortgage?
  - actual output: 

A manufacturer of disk drives for notebook computers wants an MTBF of at least $50,000$ hours. Recent test results for 10 units were one failure at $10,000$ hours, another at $25,000$ hours, and two more at $45,0
  - expected output: To apply for a mortgage, you need to provide proof of income, a credit report, and a down payment. You can start by contacting a bank or a mortgage lender.
  - context: None
  - retrieval context: ['For mortgage application you need to provide proof of income, a credit report, and a down payment']


Overall Metric Pass Rates

Answer Relevancy: 0.00% pass 

‚úì Tests finished üéâ! Run 'deepeval login' to view evaluation results on Confident AI. 
‚ÄºÔ∏è  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

#### Banking non-related question

In [59]:
question = "Who painted the Mona Lisa?"
ret = serving_function.invoke(
    path=f"/v2/models/{base_model}/infer",
    body={"inputs": [question]},
)

> 2024-11-18 08:40:50,619 [info] Invoking function: {"method":"POST","path":"http://nuclio-llm-monitoring-llm-server.default-tenant.svc.cluster.local:8080/v2/models/google-gemma-2b/infer"}


In [60]:
print(ret['outputs'][0])



What does the narrator's act of snatching the quilts from Dee show about her personal values?

In the sentences below, draw a line through any incorrect verb form and write the correct past or past participle above it. If a sentence is already correct, write $C$ after it. We had sat in the stadium for more than five hours.




In [61]:
test_case2 = LLMTestCase(
    input=question,
    actual_output=ret['outputs'][0],
    expected_output="As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?",
    retrieval_context=["This is a banking agent that allowed to talk on banking related issues only."]
)

answer_relevancy_metric2 = AnswerRelevancyMetric(threshold=0.5)

results2 = evaluate(test_cases=[test_case2], metrics=[answer_relevancy_metric2])

‚ú® You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà|100% (1/1) [Time Taken: 00:07,  7.80s/test case]



Metrics Summary

  - ‚ùå Answer Relevancy (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because none of the statements in the actual output addressed the input question about who painted the Mona Lisa, and all statements provided were irrelevant., error: None)

For test case:

  - input: Who painted the Mona Lisa?
  - actual output: 

What does the narrator's act of snatching the quilts from Dee show about her personal values?

In the sentences below, draw a line through any incorrect verb form and write the correct past or past participle above it. If a sentence is already correct, write $C$ after it. We had sat in the stadium for more than five hours.


  - expected output: As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?
  - context: None
  - retrieval context: ['This is a banking agent that allowed to talk on banking related issues only.']


Overall Metric Pass Rates

Answer Rel

‚úì Tests finished üéâ! Run 'deepeval login' to view evaluation results on Confident AI. 
‚ÄºÔ∏è  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

<a id="orpo-fine-tuning"></a>
## 4. ORPO Fine-tuning

To fine-tune the model, take the requests sent to the model (questions related to and not related to banking), build a dataset according to the [ORPO](https://huggingface.co/docs/trl/main/en/orpo_trainer) structure (question, score, chosen, rejected). (Afterwards), and re-train the model with it.

The result in a fine-tuned model that only answers banking-questions.

### 4.1. Build the Training Set

First will fetch the data collected by the model monitoring from the initial traffic to the model:

In [30]:
datasets = project.list_artifacts(kind="dataset")
ds_key = datasets[0]["spec"]["db_key"]
input_ds = f"store://datasets/{project.name}/{ds_key}"

Now, we can use OpenAI ChatGPT to generate expected outputs for us (you can see the function [here](./src/generate_ds.py)).

In [27]:
ret = project.run_function(
    function="generate-ds",
    handler="generate_ds",
    params={"input_ds": input_ds},
    outputs=["new-train-ds", "dataset"],
)

> 2024-09-16 13:20:48,127 [info] Storing function: {"db":"http://mlrun-api:8080","name":"generate-ds-generate-ds","uid":"c6c242bdb57d4bcc8efdf6213eb3a313"}
> 2024-09-16 13:20:48,396 [info] Job is running in the background, pod: generate-ds-generate-ds-vwnk5
> 2024-09-16 13:25:07,957 [info] OpenAI client created
> 2024-09-16 13:25:07,999 [info] Input dataset fetched
> 2024-09-16 13:32:46,056 [info] score, chosen and rejected populated
> 2024-09-16 13:32:46,119 [info] Dataframe logged
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Uploading the dataset shards: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 1/1 [00:00<00:00,  2.58it/s]
> 2024-09-16 13:32:47,223 [info] Dataset uploaded to HF
> 2024-09-16 1

> 2024-09-16 13:32:56,821 [info] Run execution finished: {"name":"generate-ds-generate-ds","status":"completed"}


In [39]:
ret.outputs

{'dataset': 'mlrun/banking-orpo-new',
 'new-train-ds': 'store://artifacts/mm-demo-edmond/generate-ds-generate-ds_new-train-ds:latest@b7dadc94375e41429c9ad913ec26d89c'}

Now we have a new dataset for the model tuning stored in HuggingFace.

### 4.2. Fine-tune the Model

Now, we'll fine-tune the model using the ORPO algorithm, so that the model only answers the banking-related questions.

[ORPO](https://arxiv.org/abs/2403.07691) is a new method designed to simplify and improve the process of fine-tuning language models to align with user preferences.


In [ ]:
project.run_function(
    function="train",
    params={
        "dataset": "mlrun/banking-orpo-opt",
        "base_model": "google/gemma-2b",
        "new_model": "mlrun/gemma-2b-bank-v0.2",
        "device": "cuda:0",
    },
    handler="train",
    outputs=["model"],
)

### 4.3. Check the Performance of the Fine-tuned Model

Now load and deploy the trained model to see how it performs.

In [59]:
serving_function.add_model(
    base_model,
    class_name="LLMModelServer",
    llm_type="HuggingFace",
    model_name="google/gemma-2b",
    adapter="mlrun/gemma-2b-bank-v0.2",
    model_path=f"store://models/{project.name}/{base_model}:latest",
    generate_kwargs={
        "do_sample": True,
        "top_p": 0.9,
        "num_return_sequences": 1,
        "max_length": 80,
    },
    device_map="cuda:0",
)
serving_function.set_tracking()

In [60]:
deployment = serving_function.deploy()

> 2024-11-04 10:22:58,818 [info] Starting remote function deploy
2024-11-04 10:22:59  (info) Deploying function
2024-11-04 10:22:59  (info) Building
2024-11-04 10:22:59  (info) Staging files and preparing base images
2024-11-04 10:22:59  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-11-04 10:22:59  (info) Building processor image
2024-11-04 10:27:50  (info) Build complete
2024-11-04 10:31:04  (info) Function deploy complete
> 2024-11-04 10:31:13,099 [info] Successfully deployed function: {"external_invocation_urls":["llm-monitoring-edmond-llm-server.default-tenant.app.llm-3-6-0.iguazio-cd1.com/"],"internal_invocation_urls":["nuclio-llm-monitoring-edmond-llm-server.default-tenant.svc.cluster.local:8080"]}


In [ ]:
import time
for i in range(20):
    question_model(
        questions=example_questions,
        serving_function=serving_function,
        base_model=base_model,
    )
    time.sleep(3)

The Grafana model monitoring page shows a high pass rate and a high guardrails score:

![](./images/grafana_after.png)

### 4.4 Evaluate the model using DeepEval

Again, let's test the fine tuned model's performance using DeepEval:

#### Banking related question

In [73]:
question = "What is a mortgage?"
ret = serving_function.invoke(
    path=f"/v2/models/{base_model}/infer",
    body={"inputs": [question]},
)

> 2024-11-04 10:35:05,576 [info] Invoking function: {"method":"POST","path":"http://nuclio-llm-monitoring-edmond-llm-server.default-tenant.svc.cluster.local:8080/v2/models/google-gemma-2b/infer"}


In [74]:
print(ret['outputs'][0])

A mortgage is a loan used to purchase a house or other real estate. When you take out a mortgage, you agree to repay a lender, such as a bank or mortgage company, over a specified period, plus interest.


In [75]:
test_case1 = LLMTestCase(
    input=question,
    actual_output=ret['outputs'][0],
    expected_output="A mortgage is a loan used to purchase a house or other real estate.",
    retrieval_context=["A mortgage is a banking related term"]
)

answer_relevancy_metric1 = AnswerRelevancyMetric(threshold=0.5)

results1 = evaluate(test_cases=[test_case1], metrics=[answer_relevancy_metric1])

‚ú® You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà|100% (1/1) [Time Taken: 00:04,  4.84s/test case]



Metrics Summary

  - ‚úÖ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response is perfectly relevant and directly addresses the question without any irrelevant information. Great job!, error: None)

For test case:

  - input: What is a mortgage?
  - actual output: A mortgage is a loan used to purchase a house or other real estate. When you take out a mortgage, you agree to repay a lender, such as a bank or mortgage company, over a specified period, plus interest.
  - expected output: A mortgage is a loan used to purchase a house or other real estate.
  - context: None
  - retrieval context: ['A mortgage is a banking related term']


Overall Metric Pass Rates

Answer Relevancy: 100.00% pass rate




‚úì Tests finished üéâ! Run 'deepeval login' to view evaluation results on Confident AI. 
‚ÄºÔ∏è  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

#### Banking non-related question

In [76]:
question = "Who painted the Mona Lisa?"
ret = serving_function.invoke(
    path=f"/v2/models/{base_model}/infer",
    body={"inputs": [question]},
)

> 2024-11-04 10:38:48,471 [info] Invoking function: {"method":"POST","path":"http://nuclio-llm-monitoring-edmond-llm-server.default-tenant.svc.cluster.local:8080/v2/models/google-gemma-2b/infer"}


In [77]:
print(ret['outputs'][0])

As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?


In [78]:
test_case2 = LLMTestCase(
    input=question,
    actual_output=ret['outputs'][0],
    expected_output="As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?",
    retrieval_context=["This is a banking agent that allowed to talk on banking related issues only."]
)

In [79]:
answer_relevancy_metric2 = AnswerRelevancyMetric(threshold=0.5)

In [80]:
results2 = evaluate(test_cases=[test_case2], metrics=[answer_relevancy_metric2])

‚ú® You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà|100% (1/1) [Time Taken: 00:07,  7.10s/test case]



Metrics Summary

  - ‚úÖ Answer Relevancy (score: 0.5, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.50 because the response does mention the correct artist, Leonardo da Vinci, but also includes irrelevant information about being a banking agent, which detracts from the overall relevance., error: None)

For test case:

  - input: Who painted the Mona Lisa?
  - actual output: As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?
  - expected output: As a banking agent, I am not allowed to talk on this subject. Is there anything else I can help with?
  - context: None
  - retrieval context: ['This is a banking agent that allowed to talk on banking related issues only.']


Overall Metric Pass Rates

Answer Relevancy: 100.00% pass rate




‚úì Tests finished üéâ! Run 'deepeval login' to view evaluation results on Confident AI. 
‚ÄºÔ∏è  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

<a id="automated-feedback-loop"></a>
## 5. Automated Feedback Loop

The pipeline uses the `restrict_to_banking` alert to check for drift. If drift is detected, it triggers retraining of the model (using the ORPO algorithm), and then deploys the improved model.

In [11]:
%%writefile src/workflow.py
import mlrun
from kfp import dsl

    
@dsl.pipeline(
    name="LLM Feedback Loop"
)

def kfpipeline(metric_name: str, 
               input_ds):
    
    project = mlrun.get_current_project()
    
    sample = project.run_function(
        function="metric-sample",
        name="metric-sample",
        handler="sample",
        params = {"metric_name" : metric_name},
        outputs=['alert_triggered']
    )

    with dsl.Condition(sample.outputs['alert_triggered'] == "True"):

        # Generate a new DS based on the traffic
        ds = project.run_function(
            function="generate-ds",
            handler="generate_ds",
            params={"input_ds" : input_ds}, 
            outputs=["new-train-ds","dataset"])
        
        # Re-train the new model        
        train = project.run_function(
            function="train",
            params={
                "dataset": "mlrun/banking-orpo-opt",
                "base_model": "google/gemma-2b",
                "new_model": "mlrun/gemma-2b-bank-v0.2",
                "device": "cuda:0"},
            handler="train",
            outputs=["model"],
            ).after(ds)
        
        # Deploy the function with the new (re-trained) model
        deploy = project.get_function('llm-server')
        deploy.add_model(
            "google-gemma-2b",
            class_name="LLMModelServer",
            llm_type="HuggingFace",
            model_name="google/gemma-2b",
            adapter="mlrun/gemma-2b-bank-v0.2", 
            model_path=f"store://models/{project.name}/google-gemma-2b:latest",
            generate_kwargs={
                "do_sample": True,
                "top_p": 0.9,
                "num_return_sequences": 1,
                "max_length": 80,
            },
            device_map="cuda:0",
        )
        deploy.set_tracking()
        project.deploy_function("llm-server").after(train)
        

Writing genai_monit_src/workflow.py


In [12]:
project.set_function(f"db://{project.name}/llm-server")
project.set_function(f"db://{project.name}/train")
project.set_function(f"db://{project.name}/metric-sample")
project.set_function(f"db://{project.name}/generate-ds")
project.set_workflow("main", "workflow.py", embed=True)
project.save()

In [13]:
run_id = project.run(
    "main",
    arguments={"metric_name": alert_config_name, "input_ds": input_ds},
    watch=False,
)

> 2024-09-17 08:53:22,670 [info] Pipeline submitted successfully: {"id":"1012b16b-698a-4c7d-b6c3-e11b496a14d1","pipeline_name":"tutorial-main 2024-09-17 08-53-22"}
> 2024-09-17 08:53:22,671 [info] Pipeline run id=1012b16b-698a-4c7d-b6c3-e11b496a14d1, check UI for progress


> 2024-09-17 08:53:22,759 [info] Started run workflow tutorial-main with run id = '1012b16b-698a-4c7d-b6c3-e11b496a14d1' by kfp engine
